#### Scraping Trend Data

In [ ]:
import requests
from bs4 import BeautifulSoup

#Save the URL of the link that has all D1 college basketball schools to ever play
schools_url = "https://www.sports-reference.com/cbb/schools/"

#Extract the page using requests and beautiful soup from the URL
response = requests.get(schools_url)
soup = BeautifulSoup(response.content, "html.parser")

#Find and save the schools table on the page
table = soup.find("table", id="NCAAM_schools")

#Iterate through each row in the table
#tbody denotes the rows instead of rows and column names
teams = []
for row in table.find("tbody").find_all("tr"):

    #Find each cell by locating where td is in the name
    #This provides the values for each column for a given row
    td_cells = row.find_all("td")

    if td_cells:

        #Extract the first column for each row
        #This column contains the team name, which is a link that takes you to the team's page
        td = td_cells[0]  # Get the first <td> (team name)

        #Extract the link from this clickable cell
        link = td.find("a")  # Find the <a> tag inside the <td>

        #Extract the portion of the link that is the team id and save it 
        if link:
            team_url = link["href"]
            team_name = team_url.split("/schools/")[-1].split("/")[0]
            
            #Only add the team to the teams list if the to_year column is 2002 or later, meaning the team has been D1 since 2002
            try:
                to_year = td_cells[3].text.strip()  # Get the "To" year from the 4th <td> (index 3)
                if int(to_year) >= 2002:
                    teams.append(team_name)

            #Skip the team if it has not been D1 since 2002
            except ValueError:
                pass 

This first code chunk scrapes all of the team IDs for teams that have been D1 since 2002. These IDs can be used to iterate through since they are the aspects of each URL that differentiates it from other teams. Within a URL on Sports Reference, for team stats, there is a team id and a season that determine which team's stats you want to see and for what year. This above code allows me to scrape all of these team IDs so that I can iterate through them for the following loop.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment

#Create the function to scrape game data taking in a url and a team id
def scrape_game_data(url, team):

    #Store the page from the given url
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    #Extract commented out HTML that might contain hidden tables
    comments = soup.find_all(string = lambda text: isinstance(text, Comment))

    #Iterate through these commented out HTMLs and if it is a table, add it to the soup
    #This allows the function to scrape the tables even if they are not stored as a table
    for comment in comments:
        if "<table" in comment:
            soup.append(BeautifulSoup(comment, "html.parser"))
    
    #Create the table IDs using the team that you are looking for
    table_ids = [
        f"box-score-basic-{team}",
        f"box-score-advanced-{team}"
    ]

    #Create an empty object to store the data frmes
    dfs = {}

    #Iterate through the two table ids
    for table_id in table_ids:

        #Find the given table
        table = soup.find("table", {"id": table_id})

        if table:

            #Retrieve the column names by finding the "head" column
            headers = [th.text.strip() for th in table.find("thead").find_all("th")]

            #Only take the second row of headers because the first is a table name
            headers = headers[2:len(headers)]
            
            #Extract player rows
            players = [row.find("th").text.strip() for row in table.find("tfoot").find_all("tr")]
            
            # Extract the bottom row that gives team totals
            rows = []
            for row in table.find("tfoot").find_all("tr"):

                #Take each cell that gives the team totals and append them
                cells = [td.text.strip() for td in row.find_all("td")]
                if cells:
                    rows.append(cells)
            
            #Convert to dataframe with the column names as the headers extracted from above
            df = pd.DataFrame(rows, columns = headers[1:])

            #Insert player names as the first column
            df.insert(0, "Player", players)  
            dfs[table_id] = df

    #Merge the two dataframes on the player column, which would be "School Totals" as that is the only row left
    if f"box-score-basic-{team}" in dfs and f"box-score-advanced-{team}" in dfs:
        merged_df = pd.merge(dfs[f"box-score-basic-{team}"], dfs[f"box-score-advanced-{team}"], on="Player", how="inner")

        #Return the merged dataframe
        return merged_df

    else:
        return "Does not exist"

This next code chunk creates a function that will be used in the following for loop as well. The function takes in a url, specifically a url for a game, and scrapes the simple box score table and the advanced box score table for the given team. It takes the bottom row of both of these tables, which is the row for team totals, and if the function is able to find both of these tables, then it merges them together. This outputs a single row of data that has the team totals for both basic and advanced metrics for the given game. If the function is not able to find both tables, then it returns "Does not Exist".

In [ ]:
import random
import time
import numpy as np
import traceback

#Create a variable that tracks how many teams data is collected for
teams_added = 0

#Create all the empty dataframes that will be populated throughout the loop
all_tournament_games = pd.DataFrame()
all_conf_tournament_games = pd.DataFrame()
all_regular_season_end = pd.DataFrame()

#Create the seasons that are going to be scraped
seasons = list(range(2002, 2026))[::-1]
for season in seasons:

    #Begin the loop for teams. Use a try catch block so that an error won't stop the loop when we are far along
    for idx, team in enumerate(teams, start=1):
        try:
            #Print the iteration that the loop is on each time
            print(f"Iteration {idx} out of 370: Processing {season} {team}...")

            #Store the URL with the given team and season and get the page
            url = f"https://www.sports-reference.com/cbb/schools/{team}/men/{season}-schedule.html"
            response = requests.get(url)

            #If there is an issue with the URL, skip it and print that it was not found
            if response.status_code != 200 and response.status_code != 429:
                print(f"Skipping {team}, URL not found.")
                continue
            
            #If the site locks you out, print this message and stop the loop
            if response.status_code == 429:
                print("Too Many Requests. Stopping loop. Try again in 30 minutes")
                break
            
            #Store the information on the page
            soup = BeautifulSoup(response.content, "html.parser")

            #Store the schedule table from the page
            table = soup.find("table", id="schedule")

            #If no table is found, print that it wasn't found and skip the iteration
            if table is None:
                print(f"No schedule table found for {team} (Iteration {idx}). Skipping...")
                continue

            #Extract the column names
            headers = [th.text for th in table.find("thead").find_all("th")]

            #Extract the rows similar to previous scraping
            rows = []
            for tr in table.find("tbody").find_all("tr"):
                cols = [td.text.strip() for td in tr.find_all(["th", "td"])]
                if cols:  # Ignore empty rows
                    rows.append(cols)

            #Create a dataframe with all the team's scheduled games from the rows and column names
            all_games = pd.DataFrame(rows, columns = headers)

            #Add a column for team season and the conference the team is in
            all_games["team"] = team
            all_games["season"] = season
            all_games["conference"] = all_games["Conf"].mode()[0]

            #Check if the team had an NCAA tournament game, if they did not, then skip them
            if (len(all_games[all_games["Type"] == "NCAA"]) > 0):
                
                #Extract all the NCAA tournament games
                tournament_games = all_games[all_games["Type"] == "NCAA"]

                #Extract the last five games of the team's regular season
                regular_season_end = all_games[all_games["Type"] == "REG"].tail(5)
                
                #Change the game number to numeric and store it as the table's index by subtracting one
                regular_season_end["G"] = pd.to_numeric(regular_season_end["G"])
                dates = (regular_season_end["G"] - 1).tolist()

                #Find the clickable links from the date column
                date_links = soup.select("td[data-stat='date_game'] a")

                #Extract the five links from the last five regular season games
                urls = [link.get('href') for link in date_links]
                selected_urls = [urls[i] for i in dates]

                #Create the base url thta is added on to
                pre_url = "https://www.sports-reference.com/"

                #Add each url ending to the pre url to have a list of the full URLs
                selected_urls = [pre_url + su for su in selected_urls]

                #Create an empty dataframe to score last five games data
                last_five = pd.DataFrame()

                #Iterate through all five URLs
                for link in selected_urls:

                    #Use the scrape game data function to scrape the team totals for each game
                    one_game = scrape_game_data(link, team)

                    #Sleep for a random time so we are not locked out
                    time.sleep(random.uniform(2, 5))

                    #If the game table isn't found then print that it was not
                    if isinstance(one_game, str):
                        print("Game Table not Found")
                    else:
                        #Add each game to the last_five dataframe
                        last_five = pd.concat([last_five, one_game], ignore_index = True)

                #Only include this team if all five of their last games were scrapeable
                if len(last_five) != 5:
                    print("Not Scrapeable")
                    continue
                
                ##Change all the values to numeric
                last_five = last_five.apply(pd.to_numeric, errors='coerce')

                #Drop the following two columns when they exist
                #Take the mean of each column across the five games and turn this into a new dataframe
                columns_to_drop = ["Player", "GmSc"]
                five_game_average = pd.DataFrame([last_five.drop(columns = [col for col in columns_to_drop if col in last_five.columns]).mean()])

                five_game_average["team"] = team
                
                #Add the five_game average values onto the end of the tournament games
                tournament_games = pd.merge(tournament_games, five_game_average, on = "team", how = "left", suffixes = ('', '_last_five_reg'))

                #Rename the columns so they have the same suffix for each from the last_five table
                for col in five_game_average.columns:
                    if col != "team":
                        tournament_games.rename(columns = {col: col + "_last_five_reg"}, inplace = True)

                #Create strength of schedule in their last five games and how many wins they got
                tournament_games["SRS_last_five_reg"] = pd.to_numeric(regular_season_end["SRS"], errors="coerce").mean()
                tournament_games["last_five_wins"] = regular_season_end.iloc[:, 8].value_counts().get("W", 0)





                #Do the same thing with the conference tournament games if they exist
                if (len(all_games[all_games["Type"] == "CTOURN"]) > 0):

                    conf_tournament_games = all_games[all_games["Type"] == "CTOURN"]

                    conf_tournament_games["G"] = pd.to_numeric(conf_tournament_games["G"])
                    conf_tournament_dates = (conf_tournament_games["G"] - 1).tolist()

                    # Find all the clickable links in the Date column
                    conf_tournament_date_links = soup.select("td[data-stat='date_game'] a")

                    conf_tournament_urls = [conf_tournament_link.get('href') for conf_tournament_link in conf_tournament_date_links]

                    # Print the extracted URLs
                    conf_tournament_selected_urls = [conf_tournament_urls[i] for i in conf_tournament_dates]

                    conf_tournament_selected_urls = [pre_url + su for su in conf_tournament_selected_urls]

                    conf_tournament = pd.DataFrame()
                    for link in conf_tournament_selected_urls:
                        one_conf_tournament_game = scrape_game_data(link, team)
                        time.sleep(random.uniform(2, 5))
                        if isinstance(one_conf_tournament_game, str):
                            print("Conference Tournament Game Table not Found")
                        else:
                            conf_tournament = pd.concat([conf_tournament, one_conf_tournament_game], ignore_index = True)

                    if len(conf_tournament) < 1:
                        print("Not Scrapeable")
                        continue

                    conf_tournament = conf_tournament.apply(pd.to_numeric, errors='coerce')
                
                    conf_tournament_average = pd.DataFrame([conf_tournament.drop(columns=[col for col in columns_to_drop if col in conf_tournament.columns]).mean()])

                    conf_tournament_average["team"] = team

                    tournament_games = pd.merge(tournament_games, conf_tournament_average, on = "team", how = "left", suffixes = ('', '_conf_tournament'))

                    for col in conf_tournament_average.columns:
                        if col != "team":
                            tournament_games.rename(columns={col: col + "_conf_tournament"}, inplace=True)

                    tournament_games["SRS_conf_tournament"] = pd.to_numeric(conf_tournament_games["SRS"], errors="coerce").mean()
                    tournament_games["wins_conf_tournament"] = conf_tournament_games.iloc[:, 8].value_counts().get("W", 0)
                    tournament_games["conf_tournament_champ"] = 1 if conf_tournament_games.iloc[:, 8].value_counts().get("L", 0) == 0 else 0

                    #Combine the conference tournament games and the last five regular season games into one dataframe
                    season_end = pd.concat([conf_tournament, last_five], ignore_index = True)

                    #Calculate the mean of all of these games 
                    season_end_average = pd.DataFrame([season_end.drop(columns=[col for col in columns_to_drop if col in season_end.columns]).mean()])

                    season_end_average["team"] = team

                    #Merge them into the tournament games
                    tournament_games = pd.merge(tournament_games, season_end_average, on = "team", how = "left", suffixes = ('', '_season_end'))

                    #Add a suffix
                    for col in season_end_average.columns:
                        if col != "team":
                            tournament_games.rename(columns={col: col + "_season_end"}, inplace=True)

                    #Calculate the amount of wins and losses the team had in their last five regualar season and conference tournament games
                    tournament_games["wins_season_end"] = tournament_games["wins_conf_tournament"] + tournament_games["last_five_wins"]
                    tournament_games["losses_season_end"] = np.where(tournament_games["conf_tournament_champ"] == 1, 
                                                                (5 - tournament_games["last_five_wins"]), 
                                                                (6 - tournament_games["last_five_wins"]))

                    #Print the amount of rows of the full data frame before appending
                    print(f"Rows before appending: {len(all_tournament_games)}")
                    if not tournament_games.empty:

                        #Append the tournament games to all the tournament games from past iterations
                        all_tournament_games = pd.concat([tournament_games, all_tournament_games], ignore_index=True)

                        #Pring the amount of rows after the iteration as well as the number of teams that hvae been added
                        teams_added = teams_added + 1
                        print(f"Rows after appending: {len(all_tournament_games)}")
                        print(teams_added, "Teams have been added")

                #If for some reason, the team did not play in a conference tournament do the following
                else:
                    
                    #Create a dataframe of zeros with the team column being the only one filled
                    conf_tournament_average = pd.DataFrame([0], columns = conf_tournament_average.columns)
                    conf_tournament_average["team"] = team

                    #Merge this zero dataframe to the tournament games
                    tournament_games = pd.merge(tournament_games, conf_tournament_average, on = "team", how = "left", suffixes = ('', '_conf_tournament'))

                    #Add the suffix
                    for col in conf_tournament_average.columns:
                        if col != "team":
                            tournament_games.rename(columns={col: col + "_conf_tournament"}, inplace=True)
                    
                    #Make all the other columns zero as well
                    tournament_games["SRS_conf_tournament"] = 0
                    tournament_games["wins_conf_tournament"] = 0
                    tournament_games["conf_tournament_champ"] = 0

                    #The following is the same as above, but now the season_end dataframe will only be the last five regualr season games since there were no conference tournament games
                    season_end = last_five.copy()

                    season_end_average = pd.DataFrame([season_end.drop(columns=[col for col in columns_to_drop if col in season_end.columns]).mean()])

                    season_end_average["team"] = team

                    tournament_games = pd.merge(tournament_games, season_end_average, on = "team", how = "left", suffixes = ('', '_season_end'))

                    for col in season_end_average.columns:
                        if col != "team":
                            tournament_games.rename(columns={col: col + "_season_end"}, inplace=True)

                    tournament_games["wins_season_end"] = tournament_games["wins_conf_tournament"] + tournament_games["last_five_wins"]
                    tournament_games["losses_season_end"] = np.where(tournament_games["conf_tournament_champ"] == 1, 
                                                                (5 - tournament_games["last_five_wins"]), 
                                                                (6 - tournament_games["last_five_wins"]))

                    print(f"Rows before appending: {len(all_tournament_games)}")
                    if not tournament_games.empty:
                        all_tournament_games = pd.concat([tournament_games, all_tournament_games], ignore_index=True)
                        print(f"Rows after appending: {len(all_tournament_games)}")



            #Sleep for 2 to 5 seconds before scraping again to not get locked out and print how long it slept for
            sleep_time = random.uniform(2, 5)
            print(f"Iteration {idx}: Sleeping for {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)

        #Output any error that happened while scraping a given iteration
        except Exception as e:
            print(f"Error before printing: {e}")
            traceback.print_exc()
            print("Skipping to the next team...\n")
            continue

    


Iteration 1 out of 8880: Processing 2002 duke...
Error before printing: list index out of range
Skipping to the next team...

Iteration 1 out of 8880: Processing 2003 duke...
Error before printing: list index out of range
Skipping to the next team...

Iteration 1 out of 8880: Processing 2004 duke...


Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 72, in <module>
    selected_urls = [urls[i] for i in dates]
                     ~~~~^^^
IndexError: list index out of range
Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 72, in <module>
    selected_urls = [urls[i] for i in dates]
                     ~~~~^^^
IndexError: list index out of range


Error before printing: list index out of range
Skipping to the next team...

Iteration 1 out of 8880: Processing 2005 duke...


Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 72, in <module>
    selected_urls = [urls[i] for i in dates]
                     ~~~~^^^
IndexError: list index out of range
/tmp/ipykernel_1562/228980485.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conf_tournament_games["G"] = pd.to_numeric(conf_tournament_games["G"])
Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 125, in <module>
    conf_tournament_selected_urls = [conf_tournament_urls[i] for i in conf_tournament_dates]
                                     ~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range


Error before printing: list index out of range
Skipping to the next team...

Iteration 1 out of 8880: Processing 2006 duke...


...

Rows before appending: 15
Error before printing: Reindexing only valid with uniquely valued Index objects
Skipping to the next team...

Iteration 1 out of 8880: Processing 2016 duke...


Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 177, in <module>
    all_tournament_games = pd.concat([tournament_games, all_tournament_games], ignore_index=True)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/reshape/concat.py", line 395, in concat
    return op.get_result()
           ^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/reshape/concat.py", line 680, in get_result
    indexers[ax] = obj_labels.get_indexer(new_labels)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3885, in get_indexer
    raise InvalidIndexError(self._requires_unique_msg)
pandas.errors.InvalidIndexError: Reindexing only valid with uniquely valued Index objects
/tmp/ipykernel_1562/228980485.py:116: SettingWithCopy

Rows before appending: 15
Error before printing: Reindexing only valid with uniquely valued Index objects
Skipping to the next team...

Iteration 1 out of 8880: Processing 2017 duke...


Traceback (most recent call last):
  File "/tmp/ipykernel_1562/228980485.py", line 177, in <module>
    all_tournament_games = pd.concat([tournament_games, all_tournament_games], ignore_index=True)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/reshape/concat.py", line 395, in concat
    return op.get_result()
           ^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/reshape/concat.py", line 680, in get_result
    indexers[ax] = obj_labels.get_indexer(new_labels)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/codespace/.local/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3885, in get_indexer
    raise InvalidIndexError(self._requires_unique_msg)
pandas.errors.InvalidIndexError: Reindexing only valid with uniquely valued Index objects
/tmp/ipykernel_1562/228980485.py:116: SettingWithCopy

KeyboardInterrupt: 

The above chunk of code does the actual scraping of our trend variables. The code first scrapes a given team and season url, using the team ids scraped from before. With this link, we create a table of all the team's games, subset the NCAA tournament games (the ones we will be predicting), conference tournament games, and last five regular season games. For the conference tournament and last five games, we used the function created previously on each game in both dataframes. This gives the team totals for each game entered in. When we have a dataframe of team totals from all the games, we calculate the average across them all and merge this to the NCAA tournament games. We do the same thing with the conference tournament and last five regular season games combined. Now, we have a row for each NCAA tournament game since 2002, with data on the team's performance in the last five regular season games, conference tournament games, and both combined.

#### Scraping Team Season-Long Data

In [ ]:
all_season_stats = pd.DataFrame()

#Now scrape through the seasons only since the given URL has stats for each team
for season in seasons:
  
  #Store the url with the given season
  url = f"https://www.sports-reference.com/cbb/seasons/men/{season}-school-stats.html"

  #Headers to mimic a real browser request
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

  #Retrieve the page from the link
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")

  #Find the basic stats table on this link
  table = soup.find("table", {"id": "basic_school_stats"})

  #Extract the rows from the basic stats table
  rows = []
  team_codes = []
  for tr in table.find("tbody").find_all("tr"):
      row = [td.text.strip() for td in tr.find_all("td")]

      #Extract the team id for merging from the link that is in the team column
      school_td = tr.find("td", {"data-stat": "school_name"})
      if school_td:
          team_link = school_td.find("a")
          if team_link:
              
              #Extract the actual link
              team_url = team_link["href"]

              #Find the team code in the url
              team_code = team_url.split("/")[-3]
          else:
              
              #If either wasn't found, then make the team_code column blank
              team_code = None
      else:
          team_code = None
      
      #Append the rows and team codes
      if row:
          rows.append(row)
          team_codes.append(team_code)

  #Extract the number of columns and headers
  num_cols = len(rows[0])
  headers = [th.text.strip() for th in table.find("thead").find_all("th")]

  #Ensure header count matches the data
  if len(headers) != num_cols:
      headers = ["Column_" + str(i) for i in range(num_cols)]

  #Create a dataframe from the rows and column names
  basic = pd.DataFrame(rows, columns = headers)
  
  #Store the actual column names
  col_names = ["School", "G", "W", "L", "W/L%", "SRS", "SOS", "blank1", "conf_w", "conf_l", "blank2", "home_w", "home_l", "blank3", "away_w", "away_l", "blank4", "team_points", "opponent_points", "blank", "mp", "fg", "fga",
              "fgper", "3p", "3pa", "3p_per", "ft", "fta", "ftper", "orb", "trb", "ast", "stl", "blk", "tov", "pf"]
  
  #Assign the column names to the actual column names
  basic.columns = col_names

  #Extract the schools that have NCAA in their school column, indicating they were in the NCAA tournament
  basic = basic[basic['School'].str.contains('NCAA', case=False, na=False)]
  
  #Remove this NCAA from each of these school names
  basic['School'] = basic['School'].str.replace('NCAA', '', regex=False)

  basic["Season"] = season



  
  #Do the same thing, but now for the advanced school stats
  url = f"https://www.sports-reference.com/cbb/seasons/men/{season}-advanced-school-stats.html"

 
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
  }

 
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")

 
  table = soup.find("table", {"id": "adv_school_stats"})


  rows = []
  team_codes = []
  for tr in table.find("tbody").find_all("tr"):
      row = [td.text.strip() for td in tr.find_all("td")]


      school_td = tr.find("td", {"data-stat": "school_name"})
      if school_td:
          team_link = school_td.find("a")
          if team_link:
              team_url = team_link["href"]
              team_code = team_url.split("/")[-3] 
          else:
              team_code = None 
      else:
          team_code = None 

      if row:
          rows.append(row)
          team_codes.append(team_code)


  num_cols = len(rows[0])
  headers = [th.text.strip() for th in table.find("thead").find_all("th")]

  if len(headers) != num_cols:
      headers = ["Column_" + str(i) for i in range(num_cols)]

  advanced = pd.DataFrame(rows, columns=headers)

  col_names = ["School", "G", "W", "L", "W/L%", "SRS", "SOS", "blk1", "conf_w", "conf_l", "blk2", "home_w", "home_l", "blk3", "away_w", "away_l", "blk4", "pts_scored", "pts_allowed", "blk5", "pace", "ortg", "FTr",
              "3PAr", "true_shooting", "rebound_percentage", "assist_percentage", "steal_percentage", "block_percentage", "eFG", "tov_percentage", "oreb_percentage", "ft_per_fga"]

  advanced.columns = col_names

  advanced["team_code"] = team_codes

  advanced = advanced[advanced['School'].str.contains('NCAA', case=False, na=False)]

  advanced['School'] = advanced['School'].str.replace('NCAA', '', regex=False)

  
  #Merge the advanced data and basic data on school
  one_season = pd.merge(basic, advanced, on=['School'], how = "left")

  #Combine the season stats to all previous iterations
  all_season_stats = pd.concat([all_season_stats, one_season])

  #Output the season that has been scraped
  print(f"Scraped {season} Season")
  
  #Sleep for a random time between 2 and 5 seconds and print the length
  sleep_time = random.uniform(2, 5)
  print(f"Sleeping for {sleep_time:.2f} seconds...")
  time.sleep(sleep_time)

The above code scrapes season long data for each team in the given seasons. It scrapes basic stats along with advanced stats and puts them together. The data also scrapes each team's team id so that it can be easily merged with out other dataframe.

#### Merging

In [ ]:
#Merge the all tournament games with the trend data with the season long stats data
#Merge on team and season with a left join so that we only include teams that we have games for
merged_data = pd.merge(all_tournament_games, all_season_stats, left_on = ["team", "season"], right_on = ["team_code", "Season"], how = "left")

#Create a column that combines the date and time 
merged_data["date_time"] = merged_data["Date"] + " " + merged_data["Time"]

#Use this column to merge the data with itself, adding a suffix
expanded_merged = merged_data.merge(merged_data, on = "date_time", suffixes = ("", "_opponent"), how = "left")

#Remove the rows where the team is "playing itself"
expanded_merged = expanded_merged[expanded_merged["team"] != expanded_merged["team_opponent"]]

#Output the data to a csv
expanded_merged.to_csv("merged_data.csv")

This above code merges our two data frames together. At that point, we have two rows for each game. In order to make only one row for each game, we merge the data to itself, combining all games to themselves. In this step, we add an opponent suffix so that you know those columns are for the opponent. From here, we remove all the rows where the team joined with itself. 

At this point, we have our final dataset. This dataset has a row for each NCAA tournament game since 2002 that we were able to scrape. Each row has a random team as the "point of view" team, along with their stats, their opponent's stats, and whether or not they won the game (our response variable)